## Manual Pipeline

In the previous notebook `0_simulation.ipynb` we simulated dataset. Our simulated dataset includes __detections__ and __resection_mni__. Now we are ready to run `Manual detections pipeline` using __detections__ as manually selected spikes and compare the outlut to the __resection_mni__ (ground truth). In this notebook we have the following goals:
1. Prepare and run irritative zone delineation using __detections__ timepoints.
2. Evaluate the quality of the irritative zone prediction using __resection__.

We will use the results presented in this nodebook as a baseline for the automated pipelines.

First, we import the simulation module and other packages. NOTE: To import the simulation we should change the working directory if we are running this example from the cloned GitHub repository.

In [ ]:
import os
from pathlib import Path

import matplotlib.pylab as plt
import mne
import numpy as np

from nilearn import plotting
import nibabel as nb

# change to the root directory of the project
if os.getcwd().split("/")[-1] == "examples":
    os.chdir('..')

from megspikes.simulation.simulation import Simulation

# Setup the path for the simulation
sample_path = Path(os.getcwd()) / 'examples' / 'data'
sample_path.mkdir(exist_ok=True, parents=True)

%load_ext autoreload
%autoreload 2

## Prepare the simulation

In [ ]:
sim = Simulation(sample_path)
sim.simulate_dataset()
sim

`sim.detections` is an array of spike peak localizations in samples. We will use it as manual detections.

In [ ]:
sim.detections

`sim.clusters` is an array of clusters that corresponds to the simulation source index.

In [ ]:
sim.clusters

## Run Manual Pipeline

In [ ]:
from megspikes.pipeline import manual_pipeline
from megspikes.database.database import read_meg_info_for_database
from sklearn import set_config
set_config(display='diagram')

In [ ]:
db = read_meg_info_for_database(
    sim.case_manager.fif_file,
    sim.case_manager.fwd['ico5'])

In [ ]:
pipe = manual_pipeline(sim.case_manager, db, 
                       sim.detections, sim.clusters - 1)
pipe

In [ ]:
dataset, meg_data = pipe.fit_transform((None, sim.raw_simulation.copy()))

`dataset` is an instance of xarray.Dataset, and it includes all the results. We can explore the results and the corresponding metadata.

In [ ]:
dataset

## View localized cluster

First we should convert numpy ndarray to mne.SourceEstimate. We use `array_to_stc` for this.

In [ ]:
from megspikes.localization.localization import array_to_stc

stc = array_to_stc(dataset.mne_localization.values[0, 0, :, :],
                   sim.case_manager.fwd['ico5'],
                   sim.case_manager.case)

Now we can plot the cluster.

In [ ]:
%matplotlib qt5
brain = stc.plot(subjects_dir=sim.case_manager.freesurfer_dir, hemi='both')